Для прогнозирования используются данные о покупках юзеров за период 2021 года. Всего доступно больше 2 млн. строк.

Флаг оттока проставляется, если после покупки юзера прошло больше 45 дней.

Поведение юзера описывается историей его покупок, суммой чека и размером скидки к чеку. В компании действует программа лояльности, участвуя в которой юзеры получают скидку на любую покупку и могут получать дополнительные скидки по разным промо-акциями.

Описание данных:
data.csv
* `'clnt_ID'` - уникальный айди юзера, str
* `'timestamp'` - дата и время совершения покупки, datetime
* `'gest_Sum'` - сумма покупки, float
* `'gest_Discount'` - сумма скидки, float

target.csv
* `'clnt_ID'` - уникальный айди юзера, str
* `'target'` - флаг оттока, int: 1 если юзер ушел в отток | 0 если НЕ отток

## Что здесь можно сделать? - Заметки 
**Нужно помнить, что по сути это тайм серия.**  
Т.е. я предполагаю, что клиент покупает, покупает, покупает, потом фигак и перестал покупать. Один клиент - один вектор.
А тут куча вектором принадлежат одному клиенту. 

Т.е. я бы даже сказал, что в том виде в котором оно есть, модель особо-то ничего не найдет. Как она сможет по дате, сумме покупки и скидке предсказать уйдет человек или нет, не зная его истории?

**Что может говорить, что клиент собрался уйти и не вернуться?**  
Т.е. гипотетически что может говорить о том, что клиент собирается уйти и не вернуться?
1. Увеличивается интервал между покупками от покупки к покупке. Типа 1 день, потом 2, потом 5 и т.д.
2. Как-то изменяется объем чека (уменьшается, увеличивается?) 
3. Как-то меняется ассортимент (типа вот он покупал одно и тоже, а тут вдруг перестал покупать одно и тоже (испортился товар на его вкус) и он либо сразу ушел, либо попробовал другое и остался, либо попробовал другое - ему не понравилось - ушел

**Как должен работать предикт? Какие данные должны поступать на вход?**  
Это видимо тоже относится к тому, что это своего рода таймсерия

**Если сейчас дать модели на вход просто дату, сумму чека и скидку...**  
То она будет пытаться предсказать отток исключительно по дате, сумме чеку и скидке - врядли у нее что-то получится взразумительное.


**Сумма в месяц стабильна для юзера, а отклонение может указывать на отток**  
По результатам уже проведенного анализа есть такой вывод:
сумма, которую готов тратить юзер в месяц,  достаточна стабильна и слабо меняется со временем. Потратив в этом месяце больше обычного, юзер скорее всего в следующем не будет покупать;

По каждому клиенту взять средний чек, каким-то образом определить порог отклонения от этого среднего чека и посмотреть насколько факт этого отклонения коррелирует с оттоком. Может это будет хорошая фича. Но проблема в том, что для предсказания не достаточно 

**Самый простой и очевидный способ подготовки данных**  
Это взять по каждому клиенту кол-во покупок, min,max,mean,median,stdev по gest_Sum, gest_Discount, и перерывом между покупками

**Вариант еще лучше!**  
Используем каждую строку как есть, не переводим ее в строку по уникальному клиенту, а точно также одна строка одна сделка.
К каждой такой строке мы накопительно добавляем кумулятивную инфу о клиенте, т.е.:
* Сколько дней на момент сделки прошло с момента первой сделки
* Какой на текущий момент средний чек
* Какая разница между среднем чеком и чеком сделки
* Сколько сделок произошло на момент текущей сделки
* Сколько сделок в месяц в среднем

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# pd.set_option('display.max_rows', 100)

In [2]:
df = pd.read_csv('data/data.csv')
df_target = pd.read_csv('data/target.csv')

In [ ]:
display(df.head())
df_target.head()

## Предобработка датасетов

In [ ]:
df.info()

In [ ]:
df_target.info()

* timestamp - object, а не datetime

Изменим тип данных в timestamp на datetime

In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

Проверим на дубликаты и пропуски.

In [ ]:
df.isna().sum()

Пропусков нет.

In [ ]:
df.duplicated().sum()

Есть дубликаты, посмотрим на них.

In [ ]:
df.loc[df.duplicated(keep=False)]

Удалим дубликаты.

In [4]:
df = df.drop_duplicates()

In [5]:
df.duplicated().sum()

0

Дубликаты удалили.

## Нужно создать фичи

Для анализа нужно создать фичи

**Как создавать фичи?**

* У нас есть целиковый датасет
* Нужно группировать его по клиентам
* Нужно к этим группам применять разные методы, чтобы уже эти фичи создать

## EDA

Объединим датасеты для анализа.

In [6]:

df_eda = df.merge(df_target,on='clnt_ID')
display(df_eda.head(),df_eda.info(),df_eda.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2371506 entries, 0 to 2371505
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   clnt_ID        object        
 1   timestamp      datetime64[ns]
 2   gest_Sum       float64       
 3   gest_Discount  float64       
 4   target         int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 108.6+ MB


,clnt_ID,timestamp,gest_Sum,gest_Discount,target
0,193B4268-0B4A-475E-B1D0-FF5515E29D29,2021-01-02 09:09:17.060,900.000000,300.000000,0
1,193B4268-0B4A-475E-B1D0-FF5515E29D29,2021-01-07 17:09:04.120,393.700012,131.300003,0
2,193B4268-0B4A-475E-B1D0-FF5515E29D29,2021-01-09 11:49:13.163,817.500000,272.500000,0
3,193B4268-0B4A-475E-B1D0-FF5515E29D29,2021-01-11 18:38:07.737,337.500000,112.500000,0
4,193B4268-0B4A-475E-B1D0-FF5515E29D29,2021-01-12 14:53:53.847,180.000000,60.000000,0


None

clnt_ID          0
timestamp        0
gest_Sum         0
gest_Discount    0
target           0
dtype: int64

Посмотрим кол-во уникальный айдишников.

In [7]:
display(df['clnt_ID'].nunique())
df_target['clnt_ID'].nunique()

255109

255109

255109 клиентов. При этом в датасете с таргетом тоже 255109 записей. Значит все верно.

Посмотрим сколько из них с флагом.

In [8]:
df_target['target'].value_counts()

0    128857
1    126252
Name: target, dtype: int64

Примерно одинаково. Дисбаланса классов нет.

Проверим, что все правильно смержилось.

In [11]:
(df_eda.loc[df_eda['target']==0,'clnt_ID'].nunique(),df_eda.loc[df_eda['target']==1,'clnt_ID'].nunique())

(128857, 126252)

Да, кол-во уникальных айдишников в каждом классе осталось прежним.

Посмотрим сколько заказов (строк) приходится на каждый класс.

In [ ]:
display(df_full['target'].value_counts())
sns.countplot(x='target',data=df_full)
plt.show()

Тут мы видим сразу, что те, кто уходят, значительно меньше раз совершали покупки.

### Задачи EDA

1. 

# Нужно создать фичу с кол-во дней с прошлой покупки

Как создать такую фичу?
1. Сгруппируем по юзерам
2. С помощью shift измерим разницу

In [ ]:
df.head()
df['date'] = df['timestamp'].dt.date

In [ ]:
for client, group in df.groupby('clnt_ID'):
    break
group

In [ ]:
group['date'].diff()

In [ ]:
def trans_funk(col):
#     print(col.diff())
    return col.diff()
#     print(col)
df['buy_diff_in_days'] = df.groupby('clnt_ID')['date'].transform(trans_funk)

In [ ]:
df.head()

In [ ]:
df.head(10)['timestamp'].diff()

In [ ]:
# В итоге мы пока откажемся от подхода с таймсериес, и возьмем просто описание истории покупок клиентов.
# Т.е. берем фичи сумма чека, скидка, дней с последней покупки для каждого клиента и сравнимаем эти статистики по группам 0 и 1 (таргету)
# df_full.pivot_table(index='clnt_ID', values=['gest_Sum','gest_Discount'],aggfunc=['count','min','max','median','mean','std'],columns='target')